# Will need your own Calories Ninja API Key and Google Cloud Vision API Key to run. 

In [ ]:
import pandas as pd
import requests
import os

# Google Libs
!pip install --upgrade google-cloud-vision
!pip install --upgrade google-cloud-speech
!pip install --upgrade google-cloud-language
!pip install --upgrade google-cloud-texttospeech
!pip install PyDictionary
!pip install fuzzywuzzy

In [ ]:
from google.cloud.vision_v1 import types
import google.cloud.vision_v1
from google.cloud import vision
import re
import requests

## Insert your json file path from google cloud vision API

In [ ]:
from google.colab import drive
from google.colab import files

drive.mount("/content/drive/", force_remount = True)

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [ ]:
# Imports Credential File:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "[Json file path]"
print("Service Account Key: {}".format(os.environ["GOOGLE_APPLICATION_CREDENTIALS"]))

In [ ]:
def detect_text_uri(uri):
    """Detects logos in the file located in Google Cloud Storage or on the Web.
    """
    # Imports the Google Cloud client library
    
    
    client = google.cloud.vision_v1.ImageAnnotatorClient()
    image = google.cloud.vision_v1.types.Image()
    image.source.image_uri = uri
    print(image)

    response = client.text_detection(image=image)
    texts = response.text_annotations

    return texts
    
text = detect_text_uri("https://world.openfoodfacts.org/images/products/930/060/145/1272/front_en.10.full.jpg")

source {
  image_uri: "https://world.openfoodfacts.org/images/products/930/060/145/1272/front_en.10.full.jpg"
}



In [ ]:
def get_final_content(text):

  areas = list()
  for phrase in text:
    height = phrase.bounding_poly.vertices[-1].y - phrase.bounding_poly.vertices[0].y
    width = phrase.bounding_poly.vertices[1].x - phrase.bounding_poly.vertices[0].x
    area = height * width
    areas.append(area)

  content = list()
  brand_name = None
  for phrase in text:
    content.append(phrase.description.lower())
  
  areas2ind = [(i, area) for i, area in enumerate(areas)]
  areas2ind = sorted(areas2ind, reverse = True, key = lambda x:x[1])
  kept_indices = [tup[0] for tup in areas2ind[:6]]
  reduced_content = list()
  for i in kept_indices: 
    regex = re.compile(r"\s")
    matches = regex.findall(content[i])
    if not matches:
      reduced_content.append(content[i])

  final_str = ' '.join(reduced_content)
  return final_str
  


In [ ]:
get_final_content(text)

'orange juice coles 07:01.000659 litres'

## Insert your Calories Ninja API key

In [ ]:
api_url = 'https://api.calorieninjas.com/v1/nutrition?query='
query = get_final_content(text)
response = requests.get(api_url + query, headers={'X-Api-Key': '[Calories Ninja API]'})
if response.status_code == requests.codes.ok:
    response_dict = eval(response.text)
else:
    print("Error:", response.status_code, response.text)

In [ ]:
def relay(response_dict):

  # Get the top result that is why we use zero index
  print("For a serving size of {} grams, Item {} has {} calories and {} grams \
of total fat and {} grams of sugar and {} milligrams of cholesterol".format(
            response_dict["items"][0]["serving_size_g"], 
            response_dict["items"][0]["name"], 
            response_dict["items"][0]["calories"], 
            response_dict["items"][0]["fat_total_g"], 
            response_dict["items"][0]["sugar_g"], 
            response_dict["items"][0]["cholesterol_mg"]
        ))   

In [ ]:
relay(response_dict)

For a serving size of 100.0 grams, Item orange juice has 46.4 calories and 0.1 grams of total fat and 8.3 grams of sugar and 0 milligrams of cholesterol
